### YelpDataCollection
Queries the Yelp search API to get those businesses that 'gluten_free' category for a given zip code.  
https://www.yelp.com/developers/documentation/v3/business_search

In [1]:
#-- Import Libraries
import pandas as pd
import os
import requests

# Yelp API key in secrets.py; .gitignore prevents the secrets.py from being pushed to GitHub
from secrets import yelpKey

In [39]:
#-- Prepare Request

# Zip Code to search
# Able to pass this as function in future
searchZipCode = "92104"


yelpDataForZipCode = getDataForZipcode(searchZipCode)

print('Completed')
print(yelpDataForZipCode)


Completed
{'ID': ['s0cB3tbz7Or92UDwaaghhA', 'adkvXHvlf4PfMOJ8_e8epQ', '0JmO9IUPf4qHa-lqTe8c3Q', 'ks_XOXJJmsCTqmyZZHTk8A', 'Gjs4FZmQqLTTuorCy41D3w', 'T_Zfp6uwrV2zrWBTVApCZg', 'NU2k-TpC5uqgj2yIsWGxKw', 'ddHJApklHi51XofnVVgp1g', '3N7CChoUi8dGs7wchVlsiA', 'D5Cf5YgVswOVQYvPwPul8Q', 'mqIH4Vhg6raicv-v2Bbk6w'], 'Name': ['Civico 1845', 'Paleo Treats', 'Curryosity', 'Starry Lane Bakery', 'Pavona Dolci', 'Cueva Bar', 'Red House Pizza', 'Nozaru Ramen Bar', "Ike's Love and Sandwiches", 'Monello', 'Jyoti-Bihanga'], 'Zipcode': ['92101', '92116', '92104', '92103', '92103', '92116', '92116', '92116', '92103', '92101', '92116'], 'Latitude': [32.7243133815557, 32.7621882, 32.72948, 32.7489336107359, 32.7469863891602, 32.7626951, 32.7613974055798, 32.76309, 32.748715, 32.7244334, 32.7632461], 'Longitude': [-117.168417719217, -117.1304338, -117.12927, -117.161455637243, -117.161323547363, -117.1418996, -117.146126797687, -117.1212, -117.154848, -117.1690214, -117.1216363], 'Price': ['$$', '$', 0, '$', 0, '

In [35]:
def getDataForZipcode(searchZipCode):
    '''
    
    '''

    # Source Url
    baseYelpUrl = "https://api.yelp.com/v3/businesses/search"

    # API Key passed through header
    headers = {
            'Authorization': 'Bearer %s' % yelpKey,
    }
    
    # Search Term
    searchTerm = 'gluten_free'
    
    # Dictionary stores data
    yelpData = {
        'ID': [],
        'Name': [],
        'Zipcode': [],
        'Latitude': [],
        'Longitude': [],
        'Price' : [],
        'Rating' : []
    }
    
    
    recordLimit = 50
    currentOffset = 0
    hasMoreData = True
    
    while hasMoreData == True:
           
        parameters = {
            'location': searchZipCode,
            'term': 'gluten_free',
            'limit': recordLimit,
            'radius': 5000,
            'offset': currentOffset,
            }
        
        response = requests.request('GET', baseYelpUrl, headers=headers, params=parameters)
        
        #- Check Response
        if (response.status_code == requests.codes.ok):
            
            # Get Json from Response
            responseJson = response.json()
            
            
            # Search Businesses
            for business in responseJson['businesses']:
                
                useBusiness = checkBusinessForUsage(business, searchZipCode, searchTerm)
                
                if (useBusiness == True):
                    
                    yelpData['ID'].append(business['id'])
                    yelpData['Name'].append(business['name'])
                    yelpData['Zipcode'].append(business['location']['zip_code'])
                    
                    yelpData['Latitude'].append(business['coordinates']['latitude'])
                    yelpData['Longitude'].append(business['coordinates']['longitude'])
                    
                    yelpData['Price'].append(getPriceForBusiness(business))
                    yelpData['Rating'].append(business['rating'])
                
                
        
        hasMoreData = False
    
    
    
    return yelpData


In [38]:
def checkBusinessForUsage(businessInfo, searchZipcode, searchTerm):
    ''' Determines if the business can be used in the Analysis
    
    Accepts : businessInfo (dictionary) contains the metadata for individual business 
                searchZipCode (str) zip code searching for data within
                searchTerm (str) category used with the search; ensure found
    
    Return : bool TRUE- business meets critera, able to use FALSE- unable to use business
    '''
    
    #- Check Within Search Zipcode
    businessZipCode = businessInfo['location']['zip_code']
    
#     if (businessZipCode != searchZipcode):
#         return False
    
    
    #- Check Search Term
    foundSearchTerm = False
    
    for category in businessInfo['categories']:
        
        if (category['alias'] == searchTerm):
            foundSearchTerm = True
            break
    
    
    return foundSearchTerm
     

In [29]:
def getPriceForBusiness(businessInfo):
    ''' Gets the price for a business; not all businesses contain this property within the JSON;
    when not found just uses 0.
    
    Accepts : businessInfo (dictionary) metadata on an individual business
    
    Returns : (num) value from price tag
    '''
    
    try:
        
        return businessInfo['price']
    
    except:
        return 0